https://www.sciencedirect.com/science/article/pii/S2405880720300480

Northeastern Switzerland (CHNE), western Switzerland (CHW), southern Switzerland (CHS), western Swiss Alps (CHAW), and eastern Swiss Alps (CHAE). These five regions are similar in size and have a distinct climate. 

In [34]:
import xarray as xr
import pandas as pd
from major_return_levels_bm import get_extreme_return_levels_bm
import os
import sys
sys.path.append('../Prelim_Stats')
import config
import numpy as np

#Here caluclating for Swiss Alps

In [35]:
mask_da_coarse = xr.open_dataset("swiss_mask_on_coarse_grid_latlon.nc")["swiss_mask_on_coarse_grid"]
region_mask_coarse = (mask_da_coarse == 5) #CHAW
mask_da_hr = xr.open_dataset("swiss_mask_on_hr_grid_latlon.nc")["swiss_mask_on_hr_grid"]
region_mask_hr = (mask_da_hr == 5) #CHAW

In [36]:
return_periods = [20, 50, 100]

rmse_table = pd.DataFrame(index=return_periods, columns=["COARSE", "EQM", "EQM_UNET", "DOTC", "DOTC_UNET", "QDM", "QDM_UNET"])
baseline_info_hr = {
    "OBS": {
        "nc_file": f"{config.TARGET_DIR}/RhiresD_1971_2023.nc",
        "variable_name": "RhiresD"
    },
    "EQM": {
        "nc_file": f"{config.BIAS_CORRECTED_DIR}/EQM/precip_BC_bicubic_r01.nc",
        "variable_name": "precip"
    },
    "EQM_UNET": {
        "nc_file": f"{config.BIAS_CORRECTED_DIR}/EQM/DOWNSCALED_TRAINING_QM_BC_precip_MPI-CSC-REMO2009_MPI-M-MPI-ESM-LR_rcp85_1971-2099_downscaled_gridset_r01.nc",
        "variable_name": "precip"
    },
    "DOTC": {
        "nc_file": f"{config.BIAS_CORRECTED_DIR}/dOTC/precip_temp_tmin_tmax_bicubic_r01.nc",
        "variable_name": "precip"
    },
    "DOTC_UNET": {
        "nc_file": f"{config.BIAS_CORRECTED_DIR}/dOTC/DOWNSCALED_TRAINING_DOTC_BC_precip_MPI-CSC-REMO2009_MPI-M-MPI-ESM-LR_rcp85_1971-2099_downscaled_gridset_r01.nc",
        "variable_name": "precip"
    },
    "QDM": {
        "nc_file": f"{config.BIAS_CORRECTED_DIR}/QDM/precip_BC_bicubic_r01.nc",
        "variable_name": "precip"
    },
    "QDM_UNET": {
        "nc_file": f"{config.BIAS_CORRECTED_DIR}/QDM/DOWNSCALED_TRAINING_QDM_BC_precip_MPI-CSC-REMO2009_MPI-M-MPI-ESM-LR_rcp85_1971-2099_downscaled_gridset_r01.nc",
        "variable_name": "precip"
    }
}

In [37]:
ds_hr_baselines = {name: xr.open_dataset(info["nc_file"]) for name, info in baseline_info_hr.items()}


In [ ]:

for rp in return_periods:
    # Coarse grid
    coarse_rl = []
    for i, lat in enumerate(lat_vals_coarse):
        for j, lon in enumerate(lon_vals_coarse):
            if region_mask_coarse.values[i, j]:
                ts = ds_coarse['precip'][:, i, j].values
                if np.isnan(ts).all():
                    coarse_rl.append(np.nan)
                    continue
                rl = get_extreme_return_levels_bm(
                    nc_file=f"{config.MODELS_DIR}/precip_MPI-CSC-REMO2009_MPI-M-MPI-ESM-LR_rcp85_1971-2099/precip_r01_coarse_masked.nc",
                    variable_name="precip",
                    lat=lat,
                    lon=lon,
                    block_size='365D',
                    return_periods=[rp],
                    return_all_periods=False
                )["return value"].values[0]
                coarse_rl.append(rl)
            else:
                coarse_rl.append(np.nan)
    df_coarse = pd.DataFrame({
        "lat": [lat for lat in lat_vals_coarse for _ in lon_vals_coarse],
        "lon": [lon for _ in lat_vals_coarse for lon in lon_vals_coarse],
        "COARSE": coarse_rl
    })
    df_coarse_masked = df_coarse[region_mask_coarse.values.flatten()]

    # HR grids: all
    results_hr = {name: [] for name in baseline_info_hr}
    for i, lat in enumerate(lat_vals_hr):
        for j, lon in enumerate(lon_vals_hr):
            if region_mask_hr.values[i, j]:
                for name, info in baseline_info_hr.items():
                    ds_hr_baseline = ds_hr_baselines[name] 
                    ts_hr = ds_hr_baseline[info["variable_name"]][:, i, j].values
                    if np.isnan(ts_hr).all():
                        results_hr[name].append(np.nan)
                        continue
                    rl = get_extreme_return_levels_bm(
                        nc_file=info["nc_file"],
                        variable_name=info["variable_name"],
                        lat=lat,
                        lon=lon,
                        block_size='365D',
                        return_periods=[rp],
                        return_all_periods=False
                    )["return value"].values[0]
                    results_hr[name].append(rl)
            else:
                for name in results_hr:
                    results_hr[name].append(np.nan)
    df_hr = pd.DataFrame({
        "lat": [lat for lat in lat_vals_hr for _ in lon_vals_hr],
        "lon": [lon for _ in lat_vals_hr for lon in lon_vals_hr],
        **results_hr
    })
    df_hr_masked = df_hr[region_mask_hr.values.flatten()]

    for baseline in rmse_table.columns:
        valid_mask = (~df_hr_masked["OBS"].isna()) & (~df_hr_masked[baseline].isna())
        rmse = ((df_hr_masked.loc[valid_mask, "OBS"] - df_hr_masked.loc[valid_mask, baseline]) ** 2).mean() ** 0.5
        rmse_table.loc[rp, baseline] = rmse

    valid_mask_coarse = (~df_hr_masked["OBS"].isna()) & (~df_coarse_masked["COARSE"].isna())
    rmse_table.loc[rp, "COARSE"] = ((df_hr_masked.loc[valid_mask_coarse, "OBS"] - df_coarse_masked.loc[valid_mask_coarse, "COARSE"]) ** 2).mean() ** 0.5

print("Pooled RMSE table for Swiss Alps region (region 5):")
rmse_table.to_csv("swiss_alps_region5_rmse_table.csv")

for ds in ds_hr_baselines.values():
    ds.close()

Target: (45.8654, 7.0208)
Closest grid cell: (45.8753, 7.0201)
Distance: 0.0100 degrees
Grid indices: lat_idx=1, lon_idx=8
Variable 'precip' extracted
Time series shape: (10957,)
Data range: -0.38 to 79.46
Target: (45.8654, 7.1622)
Closest grid cell: (45.8758, 7.1618)
Distance: 0.0104 degrees
Grid indices: lat_idx=1, lon_idx=9
Variable 'precip' extracted
Time series shape: (10957,)
Data range: -3.84 to 108.40
Target: (45.8654, 7.3037)
Closest grid cell: (45.8760, 7.3034)
Distance: 0.0106 degrees
Grid indices: lat_idx=1, lon_idx=10
Variable 'precip' extracted
Time series shape: (10957,)
Data range: -0.14 to 148.52
Target: (45.8654, 7.4451)
Closest grid cell: (45.8761, 7.4451)
Distance: 0.0107 degrees
Grid indices: lat_idx=1, lon_idx=11
Variable 'precip' extracted
Time series shape: (10957,)
Data range: -0.10 to 241.99
Target: (45.9643, 7.0208)
Closest grid cell: (45.9743, 7.0193)
Distance: 0.0101 degrees
Grid indices: lat_idx=2, lon_idx=8
Variable 'precip' extracted
Time series shape: (